# **Artificial Economies**
### This notebook breaks the complex problem of modeling an economy into simpler steps by focusing on what is at it's core. Us! Individuals, their jobs, their companies, and their transactions.

### ***Version 1.1***
##### I build agents to model firms and civilians. **The firms aim to maximize their profit** function (revenues - expenses) as their reward. There are 3 levels of the firms:
##### - Primary firms who harvest and produce raw materials (timber, metal, etc.)
##### - Secondary firms who handle the manufacturing of converting raw materials into products
##### - Tertiary firms who sell the final products.
##### All firms hire the civilians as laborers, aka, almost every civilian agent will have an assigned job, while 2-3% will be unemployed. The civilians have an income they receive as a function of their job at one of the firms. As their reward function, **the civilians try and maximize their utility from whatever items they purchase (food & water, housing, etc.) with their given income**. Prices are set by the firm which will seek to maximize it's profit while consumers will seek to manage their utility given what they can afford. Currently, money supply is non-existent, interest rates are 0%, and a central bank has not been implemented yet.

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import gym
from gym import spaces
import math

In [ ]:
class UtilityFunction:
    # U(x) = (b * P) (x)^a
    # x is the number of goods
    # b stretches the function vertically, increasing the amount of utility from a good
    # P is a random normally distributed personal preference from 0 to 2
    # a is the rate of decay of utility from a good, from 0 to 1
    def __init__(self, name, utility_stretch, rate_of_decay):
        self.name = name
        self.b = utility_stretch
        self.P = np.random.normal(0, 2)
        self.a = rate_of_decay
    
    def calculate_utility(self, num_goods):
        return (self.b * self.P) * (num_goods ** self.a)

In [ ]:
# class for the object being produced, either raw materials, intermediate goods, or final goods
class Good:
    def __init__(self, name, good_type, monetary_value, inputs_needed, utility_function, is_necessity):
        self.name = name
        good_types = {
            1: "raw material",
            2: "intermediate good",
            3: "final good"
        }
        self.good_type = good_types[good_type]
        if monetary_value == None:
            self.price = 1.0
        else:
            self.price = monetary_value
        self.inputs_needed = inputs_needed
        self.utility_function = utility_function
        self.is_necessity = is_necessity

    def __str__(self):
        return self.name

In [ ]:
# Class for the firm and the decisions it will make
class Firm:
    def __init__(self, name, firm_type, industry, wage, input_goods, output_good, inventory):
        """
        Parameters:
        name: name of the firm
        firm_type: 1 for primary, 2 for secondary, 3 for tertiary
        revenue_function: function that takes in the goods sold and returns the revenue
        expense_function: function that takes in the employees and returns the expense
        industry: the industry the firm is in
        wage: the wage the firm pays its employees per day
        input_goods: the goods the firm uses to produce its output good
        output_good: the good the firm produces
        inventory: a dictionary storing the current inventory of the firm and the last version of the inventory
        """
        # TODO: Implement venture capital for starting cash
        self.name = name
        self.employees = []
        self.wage = wage
        self.industry = industry
        firm_types = {
            1: "Primary",
            2: "Secondary",
            3: "Tertiary"
        }
        self.firm_type = firm_types[firm_type]
        self.input_goods = input_goods
        self.output_good = output_good
        self.inventory = inventory
        self.total_goods_produced = 0
        self.cash = [10000]
        self.current_cash = self.cash[-1]
        self.total_revenue = 0
        self.total_expense = 0
    
    def hire_employee(self, civilian):
        self.employees.append(civilian)

    def fire_employee(self, civilian):
        self.employees.remove(civilian)

    def produce(self):
        num_employees = len(self.employees)
        if self.inventory.contains(self.output_good.inputs_needed) and num_employees > 0:
            total_goods_produced += 1
            self.inventory.remove(self.output_good.inputs_needed)
            self.inventory.add(self.output_good)
            return self.output_good
        else:
            return None
    
    def close_work_day(self):
        self.total_goods_produced = 0

    def sell_goods(self):
        if self.inventory.contains(self.output_good):
            self.inventory.remove(self.output_good)
            self.cash.append(self.output_good.price + self.cash[-1])
            return self.output_good
        else:
            return None

    def pay_workers(self):
        wages_payed = 0
        for employee in self.employees:
            wages_payed += self.wage
            employee.cash += self.wage
        self.cash.append(self.cash[-1] - wages_payed)

    def set_price(self, price):
        self.output_good.price = price

    def set_wage(self, wage):
        self.wage = wage

    def calculate_daily_revenue(self):
        # iterate through each item in the inventory and calculate the revenue
        # Inventory example looks like this
        # Inventory = {
        #   "Starting Goods": [good1, good1, good2, good3],
        #   "Ending Goods": [good1, good2]
        # }
        ending_goods = self.inventory["Ending Goods"]
        starting_goods = self.inventory["Starting Goods"]
        goods_sold = [x for x in starting_goods if x not in ending_goods]
        revenue = 0
        for good in goods_sold:
            revenue += good.price
        return revenue

    def calculate_daily_expense(self):
        return len(self.employees) * self.wage
    
    def calculate_profit(self):
        total_revenue = self.calculate_daily_revenue()
        total_expense = self.calculate_daily_expense()
        return total_revenue - total_expense

In [ ]:
class Civilian:
    def __init__(self, name, job):
        self.name = name
        self.job = job
        self.cash = 100
        self.necessity_quantity = 1
        self.expenses = {
            "other": 0
        }
        self.consumer_inventory = {
            "other": 0
        }
        self.needs_met = True

    def check_needs(self):
        if self.necessity_quantity <= 0:
            self.needs_met = False
        elif self.necessity_quantity > 0:
            self.needs_met = True
    
    def buy_good(self, good):
        if self.total_money < good.cost:
            return None
        else:
            if good.name in self.expenses:
                if good.is_necessity:
                    self.necessity_quantity += 1
                self.expenses[good.name] += 1
            else:
                self.expenses[good.name] = 1
            self.total_money -= good.cost

    def consume_good(self, good):
        if good.name in self.consumer_inventory:
            self.consumer_inventory[good.name] -= 1
        else:
            return None
        if good.is_necessity:
            self.necessity_quantity -= 1

    def calculate_marginal_utility(self, good):
        marginal_utility = good.utility_function(self.expenses[good.name]) - good.utility_function(self.expenses[good.name] - 1)
        return marginal_utility

    def calculate_total_utility(self):
        total_utility = sum(expense.utility_function() for expense in self.expenses)
        return total_utility

In [ ]:
# Create all the goods
# GOOD: name, good_type, monetary_value, inputs_needed, utility_function, is_necessity
# UtilityFunction: name, utility_stretch, rate_of_decay
# Raw materials
# Timber, Food, Cotton, Iron, Silicon
timber = Good("Timber", 1, 1.0, None, UtilityFunction("Timber", 1.0, 0.5), True)
food = Good("Food", 1, 1.0, None, UtilityFunction("Food", 2.0, 0.5), True)
cotton = Good("Cotton", 1, 1.0, None, UtilityFunction("Cotton", 1.0, 0.5), True)
iron = Good("Iron", 1, 1.0, None, UtilityFunction("Iron", 1.0, 0.5), True)
silicon = Good("Silicon", 1, 1.0, None, UtilityFunction("Silicon", 1.0, 0.5), True)

# Intermediate goods
# Lumber, Cloth, Steel, Computer Chips
lumber = Good("Lumber", 2, 3.0, [timber], UtilityFunction("Lumber", 1.0, 0.5), True)
cloth = Good("Cloth", 2, 3.0, [cotton], UtilityFunction("Cloth", 1.0, 0.5), True)
steel = Good("Steel", 2, 10.0, [iron], UtilityFunction("Steel", 1.0, 0.5), True)
computer_chips = Good("Computer Chips", 2, 10.0, [silicon], UtilityFunction("Computer Chips", 1.0, 0.5), True)

# Final goods
# Furniture, Clothing, Computers
furniture = Good("Furniture", 3, 50.0, [lumber], UtilityFunction("Furniture", 1.0, 0.5), True)
clothing = Good("Clothing", 3, 50.0, [cloth], UtilityFunction("Clothing", 1.0, 0.5), True)
computers = Good("Computers", 3, 100.0, [computer_chips], UtilityFunction("Computers", 1.0, 0.5), True)

In [ ]:
# Create all the firms
# 2 of each type of firm, primary, secondary, and tertiary
# FIRM: name, firm_type, industry, wage, input_goods, output_good, inventory

# Primary firms
# Timber, Food, Cotton, Iron, Silicon
timber_firm = Firm("Timber Firm", 1, "Timber", 10.0, [], timber, {"Starting Goods": [], "Ending Goods": [timber]})
food_firm = Firm("Food Firm", 1, "Food", 10.0, [], food, {"Starting Goods": [], "Ending Goods": [food]})
cotton_firm = Firm("Cotton Firm", 1, "Cotton", 10.0, [], cotton, {"Starting Goods": [], "Ending Goods": [cotton]})
iron_firm = Firm("Iron Firm", 1, "Iron", 10.0, [], iron, {"Starting Goods": [], "Ending Goods": [iron]})
silicon_firm = Firm("Silicon Firm", 1, "Silicon", 10.0, [], silicon, {"Starting Goods": [], "Ending Goods": [silicon]})

# Secondary firms
# Lumber, Cloth, Steel, Computer Chips
lumber_firm = Firm("Lumber Firm", 2, "Lumber", 10.0, [timber], lumber, {"Starting Goods": [], "Ending Goods": [lumber]})
cloth_firm = Firm("Cloth Firm", 2, "Cloth", 10.0, [cotton], cloth, {"Starting Goods": [], "Ending Goods": [cloth]})
steel_firm = Firm("Steel Firm", 2, "Steel", 10.0, [iron], steel, {"Starting Goods": [], "Ending Goods": [steel]})
computer_chips_firm = Firm("Computer Chips Firm", 2, "Computer Chips", 10.0, [silicon], computer_chips, {"Starting Goods": [], "Ending Goods": [computer_chips]})

# Tertiary firms
# Furniture, Clothing, Computers
furniture_firm = Firm("Furniture Firm", 3, "Furniture", 10.0, [lumber], furniture, {"Starting Goods": [], "Ending Goods": [furniture]})
clothing_firm = Firm("Clothing Firm", 3, "Clothing", 10.0, [cloth], clothing, {"Starting Goods": [], "Ending Goods": [clothing]})
computers_firm = Firm("Computers Firm", 3, "Computers", 10.0, [computer_chips], computers, {"Starting Goods": [], "Ending Goods": [computers]})

firms = [timber_firm, food_firm, cotton_firm,
         iron_firm, silicon_firm, lumber_firm,
         cloth_firm, steel_firm, computer_chips_firm,
         furniture_firm, clothing_firm, computers_firm]

In [ ]:
# Create all the civilians
# CIVILIAN: name, job
# Civilian names
civilian_names = ["John", "Jane", "Bob", "Alice", "Joe", "Sally", "Mike", "Jill", "Jack", "Jenny"]
# Civilian jobs
civilian_jobs = ["Timber Firm", "Food Firm", "Cotton Firm", "Iron Firm", "Silicon Firm", "Lumber Firm", "Cloth Firm", "Steel Firm", "Computer Chips Firm", "Furniture Firm", "Clothing Firm", "Computers Firm"]
civilians = [Civilian(name, job) for name, job in zip(civilian_names, civilian_jobs)]

# Create the OpenAI Gym environment
# It should allow for multiple agents to interact with the environment
# The environment should be able to handle multiple agents interacting with each other
# The environment should be able to handle multiple agents interacting with the same firm
# The environment should be able to handle multiple agents interacting with the same good
# The environment should be able to handle multiple agents interacting with the same market

class EconomyEnv(gym.Env):
    metadata = {'render.modes': ['human']}
    def __init__(self):
        # Create the firms
        # Create the civilians
        # Create the goods
        # Create the markets
        # Create the government
        # Create the banks
        # Create the environment
        pass

    def step(self, action):
        # Take in an action
        # Calculate the reward
        # Calculate the next state
        # Calculate the done
        # Calculate the info
        pass

    def reset(self):
        # Reset the environment
        pass

    def render(self, mode='human', close=False):
        # Render the environment
        pass

    def close(self):
        # Close the environment
        pass

    def seed(self, seed=None):
        # Seed the environment
        pass

    def configure(self, *args, **kwargs):
        # Configure the environment
        pass

In [ ]:
# Create the environment
env = EconomyEnv()
# Run the environment
env.run()
# Close the environment
env.close()